## TP08 : algorithmes gloutons - corrigé

- Éléments de cours sur  :  <à venir>  
- Éléments de cours sur  :  <à venir>

Raccourcis pour Jupyter :
<ul>
    <li>Maj + Enter : exécute la cellule courante ;</li>
    <li>Esc b : crée une cellule de code au-dessous de la cellule courante ('b' pour 'below') ;</li>
    <li>Alt + Enter : exécute la cellule courante et créé une cellule de code à sa suite ;</li>
    <li>Esc + m : transforme une cellule de code (mode 'Code') en cellule de texte (mode 'Markdown') ;</li>
    <li>Esc + y : transforme cellule de texte (mode 'Markdown') en une cellule de code (mode 'Code').</li>
</ul>

## Introduction

Ce TP propose de découvrir, sur deux exemples classiques, deux algorithmes de type ***glouton***.  
Les algorithmes de type ***gloutons*** sont des algorithmes que l'on peut mettre en œuvre pour résoudre des problèmes de recherche d'optimum, et qui consistent à rechercher, à une étape donnée de l'algorithme, un *optimum local* (c'est-à-dire une *meilleure solution* vis-à-vis du problème posé) avant de passer à l'étape suivante consistant à rechercher un optimum sur un problème plus simple, *en espérant* que la suite des choix faits conduira à un optimum pour le problème global initial.  
Il existe des problèmes pour lesquels il existe un algorithme glouton fournissant l'optimum cherché, et d'autres pour lesquels il n'en existe pas ou pour lesquels il n'en a pas encore été trouvé.  
Dans le premier exemple traité (rendu de monnaie), l'algorithme glouton examiné ne donne pas systématiquement l'optimum : on en donnera un contre-exemple.  
Dans le deuxième exemple proposé (exercice 2 : choix d'activités), l'algorithme glouton examiné renvoie bien l'optimum : il est possible de le démontrer.

<div style='color:red;background-color:yellow'><b>À faire pendant le temps du TP :</b> exercice 1</div>
<ul>
    <li><u>première heure</u> :  Exercice 1 Partie A ;</li>
    <li><u>deuxième heure</u> : Exercice 1 Partie B</li>
</ul>

## Exercice 1 rendu de monnaie

Il s'agit ici de résoudre le problème d'optimisation classique suivant :  
* étant donnés un système de pièces et/ou de billets, $S=\left\{c_1, c_2, \ldots, c_n\right\}$, et une somme d'argent à rendre, $M$, déterminer les nombres de pièces ou de billets de chaque sorte, $\left(q_1, q_2, \ldots, q_n\right)$ de sorte à minimiser le nombre total de pièces et de billets utilisés.

De façon formalisée, $S$ et $M$ étant donnés, il s'agit de minimiser la somme $N=\sum_{k=1}^nq_k$ (nombre de pièces ou de billets utilisés) sous la contrainte $M=\sum_{k=1}^nq_kc_k$.

<u>Références</u> :  
* https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.57.3243&rep=rep1&type=pdf

#### Partie A

On considère ici le système de <a href ="https://www.banque-france.fr/billets/reconnaitre-et-utiliser-les-billets-et-les-pieces-en-euros/les-caracteristiques-des-billets-et-des-pieces-en-euro">pièces et de billets des pays de la zone euro</a> :

In [1]:
S = [0.01, 0.02, 0.05, 0.10, 0.20, 0.50, 1, 2, 5, 10, 20, 50, 100, 200]

In [2]:
len(S)

**Q1** On cherche à rendre une somme $M_1 = 464,47$ €.

**Q1.a** Donner, sous la forme d'un dictionnaire, `d1`, dont les clés sont les valeurs faciales des pièces ou billets utilisés et dont les valeurs sont les nombres de pièces ou billets utilisés, une façon de rendre la somme $M_1$.

***Q1 corrigé***

In [3]:
d1 = {200: 2, 50: 1, 10: 1, 2: 2, 0.20: 2, 0.05: 1, 0.02: 1}

**Q1.b** En parcourant le dictionnaire `d1` de la façon la plus appropriée :
* calculer le nombre de pièces utilisées ;
* vérifier que la somme rendue est bien égale à $M_1$.

***Q1.b corrigé***

On calcule le nombre de pièces à l'aide d'un parcours par les valeurs :

In [4]:
N = 0
for c in d1.values():
    N += c

* nombre de pièces utilisées :

In [5]:
N

On calcule la somme rendue à l'aide d'un parcours par les couples clés-valeurs :

In [6]:
s = 0
for c, v in d1.items():
    s += c * v

* somme rendue :

In [7]:
s

**Pour la suite, afin de s'abstraire du problème de la représentation approximative par des flottants, des pièces de moins de 1 euro,** on pourrait choisir de choisir comme unité le centime d'euro.  

Ceci étant, afin de rester dans le domaine des entiers et aussi pour limiter le nombre de pièces et de billets du système considéré, nous allons nous limiter aux pièces et billets de 1 euro ou plus et ne considérer que des sommes à rendre qui sont des nombres entiers d'euros.

**Q2.a** Décrire une stratégie pour rendre, en utilisant un minimum de pièces, une somme $S$ égale à un nombre entier d'euros avec le système de valeurs faciales `S1 = [1, 2, 5]`.

Appliquer cette stratégie pour rendre un somme de 68 €. On donnera la réponse sous la forme d'un dictionnaire, `d2`, comme en **Q1.a**.

***Q2.a corrigé***

L'algorithme intuitif (on l'a déjà appliqué à la question 1) consiste à déterminer le plus grand nombre de billets de 5 euros utilisables, puis, pour le reliquat, le plus grand nombre de pièces de 2 € possible, et terminer en rendant le reste de la somme avec des pièces de 1 €.

Suivant cette stratégie, on rendra la somme de 68 € de la façon suivante :

In [8]:
d2 = {5: 13, 2: 1, 1: 1}

* vérification

In [9]:
sum(d2.values()), sum([c * v for c, v in d2.items()])

#### Mise en œuvre de l'algorithme glouton de rendu de monnaie

On note $S = \{c_1, c_2, \ldots, c_n\}$, les système de pièces (ou de billets) utilisé, où les $c_i$ ($1\leqslant i\leqslant n$) sont des entiers, tels que $c_1=1 < c_2 < \ldots < c_n$.  

On considère l'algorithme suivant pour rendre une somme $M$, entière, en utilisant le système $S$, dans lequel on note $M_i$ la somme restant à rendre au début de l'itération d'indice $i$, en posant $M_0=M$ :  
<ul>
    <li>pour $i$ variant de $0$ à $n-1$ :</li>
    <ul>
        <li>déterminer le plus grand nombre de pièces de valeur $c_{n-i}$, noté $q_i$, pour rendre la somme $M_i$</li>
        <li>calculer la somme $M_{i+1}$ qui restera à rendre au début de l'itération suivante.
    </ul>
</ul>
Lorsque l'algorithme termine, la plus petite valeur faciale étant égale à 1 et la somme initiale à rendre, $M$, étant entière, il est assuré que la somme restant à rendre sera nulle.

**Q2.b** Compléter le squelette de fonction suivant, afin de mettre en œuvre l'algorithme proposé, sous la forme d'une fonction, prenant en entrée un système de pièces $S$ sous la forme d'une liste de valeurs entières croissante dont la plus petite vaut 1, et un entier positif $M$ représentant la somme à rendre.  
La fonction renverra les nombres de pièces de différentes valeurs utilisées *via* un dictionnaire, $d$, dont les clés sont les valeurs dans $S$ et les valeurs sont les nombres de pièces utilisées pour chaque valeur de clé.  
On utilisera les opérations de division entière et de modulo pour calculer les nombres de pièces de chaque valeur et actualiser les valeurs de la variable `M` représentant la somme restant à rendre.

In [10]:
def rendu_glouton(S, M):
    ## initialisation du dictionnaire d
    d = ...
    ## mise en œuvre de l'algorithme
    for i in range(...): ## <-- on fait varier l'indice i de 0 à len(S) - 1
        c = ... ## <--- valeur faciale de la pièce à utiliser (pour un i donné, c vaudra S[len(S) - 1 -i] )
        q = ... ## <--- calcul du nombre de pièces de valeur c à utiliser
        ...     ## <--- ajout de la clé c au dictionnaire d, associée au nombre de pièce de valeur c utilisées
        ...     ## <--- actualisation de la somme restant à rendre, M
    return d

Tester l'algorithme pour $M=68$ et le système $S_1= [1, 2, 5]$ et pour $M=464$ et le système $S_2 = [1, 2, 5, 10, 20, 50, 100, 200]$.

In [11]:
def rendu_glouton(S, M):
    ## initialisation du dictionnaire d
    d = ...
    ## mise en œuvre de l'algorithme
    for i in range(...): ## <-- on fait varier l'indice i de 0 à len(S) - 1
        c = ... ## <--- valeur faciale de la pièce à utiliser (pour un i donné, c vaudra S[len(S) - 1 -i] )
        q = ... ## <--- calcul du nombre de pièces de valeur c à utiliser
        ...     ## <--- ajout de la clé c au dictionnaire d, associée au nombre de pièce de valeur c utilisées
        ...     ## <--- actualisation de la somme restant à rendre, M
    return d

* tests

***Note :*** on peut montrer que pour les deux systèmes de pièces considérés dans les exemples le nombre total de pièces à utiliser calculé par l'algorithme est minimal. C'est aussi le cas pour les systèmes de pièces et billets utilisés dans les différents pays du monde, mais ce n'est pas assuré pour un système de pièces quelconque.

***Q2.b corrigé***

In [12]:
def rendu_glouton(S, M):
    ## initialisation du dictionnaire d
    d = {}
    ## mise en œuvre de l'algorithme
    for i in range(len(S)):
        c = S[len(S) - 1 - i] ## <--- valeur faciale de la pièce à utiliser
        q = M // c            ## <--- calcul du nombre de pièces de valeur S[n - 1- i] à utiliser
        d[c] = q              ## <--- actualisation du dictionnaire d
        M = M % c             ## <--- actualisation de la somme restant à rendre, M
        # note : M % c est équivalent à M - q * c = M - (M // c) * c
    return d

* tests 

In [13]:
rendu_glouton([1, 2, 5] , 68)

In [14]:
rendu_glouton([1, 2, 5, 10, 20, 50, 100, 200] , 464)

### Partie B &emsp; optimalité de l'algorithme glouton

L'algorithme précédemment utilisé (algorithme de type ***glouton***) n'assure pas que quel que soit le système de pièces considéré, le nombre total de pièces utilisé sera minimal.  

**Q1** Étude d'un contre-exemple

On considère ici le système de pièces $\{1, 3, 4\}$.

**Q1.a** Déterminer le nombre total de pièces utilisé si on applique l'algorithme glouton pour rendre une somme de 6 €. Vérifier que ce nombre ne correspond pas à un optimum.

In [15]:
rendu_glouton([1, 3, 4] , 6)

**Q1.b.** On cherche à déterminer toutes les sommes $M$, comprises entre $0$ et $100$, pour lequel l'algorithme glouton n'est pas optimal pour le système de pièces $\{1, 3, 4\}$.  

On va pour cela appliquer un algorithme en **force-brute** dans lequel on testera <u>toutes les façons</u> de rendre une somme $M$, $0\leqslant M\leqslant 100$, avec le système de pièces $\{1, 3, 4\}$, afin de déterminer, parmi ces valeurs de $M$, pour lesquelles l'algorithme glouton ne renvoie pas le nombre minimal de pièces utilisables.   

**Q1.b.i** Rappeler comment on peut calculer le nombre maximal pièces de valeur $c>1$ qu'il est possible d'utiliser pour rendre une somme $m$ (le reliquat étant rendu à l'aide de pièces de valeur inférieure).

***Q1.b.i corrigé***

Le nombre maximal de pièces de valeur $c$ utilisables est donné par $M // c$, quotient dans la division euclidienne de $M$ par $c$.

**Q1.b.ii** À l'aide de deux boucles imbriquées, tester tous les rendus $(q_1, q_2, q_3)$ possibles pour rendre une somme $M$ dans un système $S=\{c_1, c_2, c_3\}$.  
On suppose que $c_1=1$ et que $c_1<c_2<c_3$.  
$q_3$ désignera le nombre de pièces utilisées pour rendre la somme $m_3=M$.  
Pour $i=2$ et $i=1$, $q_i$ désignera le nombre de pièces de valeur $c_i$ pour rendre la somme $m_i$ qui reste à rendre après avoir utilisé les pièces de valeur $c_{i+1}, \ldots, c_3$.  
On comparera la somme $q_1+q_2+q_3$ au nombre total de pieces obtenu par application de l'algorithme glouton, que l'on autorise de calculer par l'expression `sum([v for v in rendu_glouton([c1, c2, c3] , M).values()])`.  
Pour chaque contre-exemple trouvé, on ajoutera le triplet $(M,\, (q_1 + q_2 + q_3, [q_1, q_2, q_3]\,)\,)$ à une liste de contre-exemples `Lcex` initialement vide.

On complètera le squelette suivant :

In [16]:
c1, c2, c3 = 1, 3, 4 ## <--- spécification du système de trois pièces à tester
Lcex = []            ## <--- initialisation de la liste de contre-exemples
for M in range(101): ## <--- on cherchera tous les rendus possibles pour les sommes M = 0, 1, ..., 100
    ## calcul préliminaire du nombre de pièces utilisées si on applique l'algorithme glouton
    nb_total_glouton = sum([v for v in rendu_glouton([c1, c2, c3] , M).values()])
    ## examen de tous les triplets (q1, q2, q3) possibles pour rendre la somme M
    m3 = M ## <--- somme restant à rendre
    for q3 in range(...): ## <--- test de toutes les nombres possibles de pièces de valeur c3 utilisées
        m2 = ... ## <--- actualisation de la somme à rendre si on utilise q3 pièces de valeur c3
        for q2 in range(...): ## <--- test de toutes les nombres possibles de pièces de valeur c2 utilisées
            q1 = ... ## <--- calcul du nombre de pièces de valeur c1 = 1 à utiliser pour rendre le reliquat
            if ...: ## <--- on teste ici si le nombre total de pièces est inférieur à celui donnée par l'algorithme glouton
                Lcex.append(...) ## <--- actualisation de la liste de contre-exemples

TypeError: 'ellipsis' object cannot be interpreted as an integer

On vérifiera que l'on retrouve bien le contre-exemple trouvé en **Q1.a**.

***Q1.b.ii corrigé***

On complète le squelette proposé en utilisant les mêmes éléments de calcul qu'en question **Q2.b** de la **partie A**, la différence étant qu'au lieu de ne retenir à chaque étape que le nombre maximal de pièces utilisables, on teste chaque possibilité.

In [17]:
c1, c2, c3 = 1, 3, 4 ## <--- spécification du système de trois pièces à tester
Lcex = []            ## <--- initialisation de la liste de contre-exemples
for M in range(101): ## <--- on cherchera tous les rendus possibles pour les sommes M = 0, 1, ..., 100
    ## calcul préliminaire du nombre de pièces utilisées si on applique l'algorithme glouton
    nb_total_glouton = sum([v for v in rendu_glouton([c1, c2, c3] , M).values()])
    ## examen de tous les triplets (q1, q2, q3) possibles pour rendre la somme M
    m3 = M
    for q3 in range(m3 // c3 + 1):
        m2 = m3 - q3 * c3
        for q2 in range(m2 // c2 + 1):
            q1 = m2 - q2 * c2 
            if q1 + q2 + q3 < nb_total_glouton: 
                Lcex.append((M, (q1 + q2 + q3, [q1, q2, q3])))

* examen des résultats

In [18]:
Lcex

In [19]:
len(Lcex) ## nombre de cotre-exemples trouvés

**Q2** **Implémentation récursive d'une recherche de l'optimum**

L'algorithme mis en oœuvre en **partie B** question **Q1.b.ii** est un examen exhaustif de toutes les solutions possibles, mais qui est difficile d'adapter pour un système avec plus grand nombre de pièces, plus encore pour un nombre de pièces quelconque.

**On propose ici une recherche exhaustive en procédant récursivement.**  
Pour un système de pièces $S=[c_1, c_2, \ldots, c_n]$ dans lequel $c_1=1<c_2<\ldots<c_n$, on s'appuie sur le fait que l'optimum, pour une somme $M$, se trouve parmi les nombres totaux minimaux de pièces obtenus, testant tous les nombres de pièces de valeur $c_n$ utilisables.  
Pour chaque nombre $q$ de pièces de valeur $c_n$ envisageable, on ajoutera à $q$, le nombre minimal, $N_q$, de pièces utilisables pour rendre le reliquat $M- q\times c_n$ avec le système de pièces $[c_1, c_2, \ldots, c_{n-1}]$, ***obtenu par un appel récursif***.  
On cherchera le minimum de toutes les sommes $q+N_q$ pour toutes les valeurs possibles de $q$.  

La fonction récursive, notée `rendu_optimal_rec`, prendra en argument un système de pièces $S$ et une somme à rendre $M$ et renverra le nombre de pièces minimal utilisable, `nb_opt`, ainsi qu'une répartition, sous la forme d'un tuple, des pièces permettant de l'obtenir, `repartition_opt`.  
Pour cette fonction récursive :  
* on considèrera comme cas de base (cas pouvant être traité directement, sans appel récursif) le cas où le système de pièces ne comporte qu'une seule pièce de valeur $1$ ;
* le cas général (se traitant à l'aide d'un (ou plusieurs) appel(s) récursifs) sera lorsque le système de pièces est de la forme $S=[c_1, c_2, \ldots, c_n]=[1, c_2, \ldots, c_n]$ avec $n>1$.  

La répartition des pièces utilisées sera renvoyée sous la forme d'une liste $[q_1, q_2, \ldots, q_n]$, nommée localement `repartition_opt`. Le nombre de pièces de valeur $c_i$, $1\leqslant i \leqslant n$, utilisées sera donnée par la valeur `repartition_opt[i - 1]`. 

In [20]:
def rendu_optimal_rec(S, M):
    ## traitement du cas de base (relativement à S, tandis que M >= 0 est quelconque)
    if S == [1]:
        return ...
    ## traitement du cas général (S = [1, ..., c_n], et M >= 0, donnée, quelconque)
    nb_opt, repartition_opt = ... # <--- initialisation du nombre minimal de pièces utilisées, nb_opt,
                                  #      et de la répartition correspondante, à la valeur la pire correspondant
                                  #      au cas où l'on utiliserait que de des pièces de valeur 1
                                  #      (initialisation nécessaire pour mettre en œuvre l'algorithme usuel
                                  #      de recherche d'un minimum pour le nombre de pièces utilisées)
    ## définition de variables pertinentes pour faciliter les appels récursifs
    c  = S[len(S) - 1]   ## c est la plus grande valeur faciale dans le système S
    S1 = S[0:len(S) - 1] ## S1 est le système de pièces S, privé de sa valeur maximale c = c_n
    for q in range(...): # on teste toutes les valeurs possibles pour le nombre q de pièces
                         # de valeur maximale c = S[len(S) - 1]
        nb_min, repartition_min = rendu_optimal_rec(...) ## <--- nombre minimal de pièces et répartition 
                                                         ##      correspondante pour rendre le reliquat
                                                         ##      à l'aide du système
                                                         ##      S[0:len(S) - 1] = [1, c_2, ..., c_(n-1)]
        if ...: # <--- on examine si la somme q + nb_min est inférieure au minimum déjà obtenun nb_opt
            nb_opt, repartition_opt = ... ## si tel est le cas, on actualise le minimum de pièces utilisable,
                                          ## nb_opt, et on enregistre la répartition de pièces correspondante
    return nb_opt, repartition_opt

* test

In [21]:
#rendu_optimal_rec([1, 3, 4], 6), rendu_glouton([1, 3, 4] , 6)

In [22]:
#rendu_optimal_rec([1, 3, 4], 38), rendu_glouton([1, 3, 4], 38)

In [23]:
S = [1, 5, 10, 25, 50]
#rendu_optimal_rec(S, 72), rendu_glouton(S, 72)

In [24]:
S = [1, 2, 5, 10, 20, 50]
#rendu_optimal_rec(S, 72), rendu_glouton(S, 72)

***Q2 corrigé***

In [25]:
def rendu_optimal_rec(S, M):
    ## traitement du cas de base (relativement à S, tandis que M >= 0 est quelconque)
    if S == [1]:
        return M, [M]
    ## traitement du cas général (S = [1, ..., c_n], et M >= 0, donnée, quelconque)
    nb_opt, repartition_opt = M, [1] + (len(S) - 1) * [0] ## initialisation (on utilise que des pièces de 1) 
    c  = S[len(S) - 1]   ## c est la plus grande valeur faciale dans le système S
    S1 = S[0:len(S) - 1] ## S1 est le système de pièces S, privé de sa valeur maximale c = c_n
    for q in range(M // c + 1): 
        nb_min, repartition_min = rendu_optimal_rec(S1, M - q * c)
        if q + nb_min < nb_opt:
            nb_opt, repartition_opt = nb_min + q, repartition_min + [q]
    return nb_opt, repartition_opt

In [26]:
rendu_optimal_rec([1, 3, 4], 6), rendu_glouton([1, 3, 4] , 6)

In [27]:
rendu_optimal_rec([1, 3, 4], 38), rendu_glouton([1, 3, 4], 38)

In [28]:
S = [1, 5, 10, 25, 50]
rendu_optimal_rec(S, 72), rendu_glouton(S, 72)

In [29]:
S = [1, 2, 5, 10, 20, 50]
rendu_optimal_rec(S, 72), rendu_glouton(S, 72)

### Exercice 2 &emsp; Choix d'activités

On présente ici un problème classique pour lequel il existe un algorithme de type *glouton* renvoyant dans tous les cas la solution optimale.

* Présentation du problème :

Au cours d'une journée de loisir, $n$ activités sont proposées, chacune sur un créneau horaire propre $[d_i, f_i[$.  
Les heures de début et de fin seront codées par des valeurs flottantes de l'intervalle $[0, 24[$.  
On cherche à déterminer le nombre maximal d'activités auxquelles une même personne peut participer pendant la journée, étant entendu qu'une même personne ne peut participer à deux activités en même temps.

* formalisation du problème

La donnée d'une instance du problème sera la donnée d'un ensemble d'activités décrit par un ensemble d'intervalles :  
$$\left\{[d_0, f_0[, [d_1, f_1[\dots, [d_{n-1}, f_{n-1}[\right\}.$$  
La résolution du problème consistera à déterminer un sous-ensemble d'indices $E = \left\{i_1, i_2, \ldots, i_p\}\right\}$ de cardinal $p$ maximal, respectant la contrainte :
$$
\forall (i, j)\in E^2, \left(i\neq j\right)\Longrightarrow \left([d_i, f_i[ \cap [d_j, f_j[ = \emptyset\right).
$$

**Q1 &emsp; Étude d'exemples**

**Q1.a** On considère le choix d'activités décrit par le diagramme suivant :

In [30]:
A = [[8., 10.], [9.5, 11.], [10., 10.5], [14., 15.5], [13., 16.]]

<img src="https://github.com/lbaille20/Info1A_2021-22/raw/main/TP08/TP08_ex2_fig01.png"/>

Déterminer le nombre maximal d'activités auxquelles une personne peut assister.

***Q1.a corrigé***

Une personne peut participer au plus à 3 activités, sur les créneaux  $\left[8., 10.\right[$,  $\left[10., 10.5\right[$ et  $\left[13., 16.\right[$.

**Q1.b** On considère la <a href="https://pageperso.lis-lab.fr/~arnaud.labourel/diu_eil/?cours9_flat#solution">situation plus complexe suivante</a> :

In [31]:
B = [[9, 12], [13, 15], [16, 21], [11, 13], [14, 18], [8, 14], [15, 19], \
     [8, 9], [13, 17], [20, 22], [12, 16], [15, 17], [10, 21]]

<img src="https://github.com/lbaille20/Info1A_2021-22/raw/main/TP08/TP08_ex2_fig02.png"/>

Estimer (*à la main*) le nombre maximal d'activités auxquelles une personne peut participer.  
Envisager si un algorithme peut se dégager de la façon dont on pense déterminer ce nombre maximal.

***Q1.b corrigé***

On peut évaluer ce nombre à 5, par le choix, par exemple, des activités associées aux créneaux $\left[8, 9\right[$, $\left[9, 12\right[$,  $\left[13, 15\right[$,  $\left[15, 17\right[$, et  $\left[20, 22\right[$.  
Selon votre façon de raisonner, il peut se dégager d'examiner les activités par heure de début croissante ou heure de fin croissante.  
On peut privilégier l'heure de fin croissante car plus une activité termine tôt, plus il est possible de trouver des activités auxquelles on peut assister ensuite.

**Q2 &emsp; Caractérisation des activités compatibles**

**Q2.a** Exprimer une condition portant sur $d_i$, $f_i$, $d_j$ et $f_j$ pour que deux activités $i$ et $j$, $i\neq j$, caractérisées par leurs horaires de début et de fin, $\left[d_i, f_i\right[$ et $\left[d_j, f_j\right[$ soient compatibles.

***Q2.a corrigé***

Deux activités sont compatibles si et seulement si l'une termine avant l'autre commence, *i.e.* $f_i\leqslant d_j$ ou $f_j\leqslant d_i$.

**Dans tout ce qui suit, une activité sera représentée par ses horaires de début et de fin.**  
Une activité $a=[d, f[$ sera représentée en machine par une liste `[d, f]`.

**Q2.b** Un certain ensemble de $k$ activités ayant été choisies, $E_k =\{i_1,, i_2, \ldots, i_k\}$, comment exprimer qu'une activité $j$ est compatible avec toutes ces activités ?

***Q2.b corrigé***

On examine si, pour tout $i\in E_k$ on a, soit $f_{i_k}\leqslant d_j$, soit $f_j\leqslant d_{i_k}$.

**Q2.c** On cherche à déterminer, au sein d'une liste d'activités donnée, les activités qui sont compatibles avec une activité donnée.

**Q2.c.i** Écrire une fonction, `activites_compatibles(L, a)`, prenant en entrée une liste d'activités $L$ et une activité $a$, et renvoyant une liste des activités de $L$ compatibles avec l'activité $a$.  
Sur l'exemple de la liste d'activités `A = [[8., 10.], [9.5, 11.], [10., 10.5], [14., 15.5], [13., 16.]]`, et pour l'activité $x=[10; 13,5[$, l'appel `activites_compatibles(A, x)` devra renvoyer la liste `[[8., 10.], [14., 15.5]]` (ou la même liste dans un ordre différent).

***Q2.c.i corrigé***

In [32]:
def activites_compatibles(L, a):
    da, fa = a
    compatibles = []
    for activite in L:
        d, f = activite
        if fa <= d or f <= da:
            compatibles.append(activite)
    return compatibles

* tests

<img src="https://github.com/lbaille20/Info1A_2021-22/raw/main/TP08/TP08_ex2_fig01.png"/>

In [33]:
A = [[8., 10.], [9.5, 11.], [10., 10.5], [14., 15.5], [13., 16.]]
activites_compatibles(A, [11., 13.])

In [34]:
A = [[8., 10.], [9.5, 11.], [10., 10.5], [14., 15.5], [13., 16.]]
activites_compatibles(A, [10., 10.5])

**Q2.c.ii** Peut-on proposer un pré-traitement de la liste $L$ de sorte que la complexité (temps de calcul) de la fonction `activites_compatibles(L, a)` soit moindre ?

***Q2.c.ii corrigé*** 

Si les activités sont ordonnées par horaire de début croissants ou horaires de fin croissants, il sera possible de ne pas tester toutes les activités pour construire la liste des activités compatibles.

**Q2.d.** 

Pour une liste d'activités donnée, $L$, proposer un algorithme récursif permettant de déterminer le nombre maximal d'activités auxquelles un individu peut participer.   
On écrira pour cela une fonction `choix_optimal(L)` prenant en argument une liste d'activités $L$ et renvoyant une liste d'activités compatibles extraite de $L$, de cardinal maximal.  
La récursion se fera sur le nombre d'activités dans la liste $L$.  
*On pourra s'inspirer de la question* ***Q2*** *de l'* ***exercice 1***, ***partie B***.

* tests : on verifiera l'optimum déterminé par la fonction sur les listes d'activités `A` et `B`

***Q2.d corrigé***

On prend comme cas de base, une liste d'activités vide, dans ce cas, la liste à renvoyer est une liste vide.  
Dans le cas général :
* on réserve la dernière activité ; 
* à l'aide d'un appel récursif, on cherche une liste de cardinal maximal d'activités compatibles entre elles parmi les activités privées la dernière activité, `choix_opt1`. La longueur de cette liste réalise à l'optimum, `nb_opt1`.  
* on recherche la liste des activités compatibles avec la dernière activité ;
* à l'aide d'un appel récursif, on cherche une liste de cardinal maximal d'activités compatibles entre elles parmi les activités compatibles avec la dernière activité, `choix_opt2`. La longueur de cette liste, augmentée de 1, est enregistrée sous le nom `nbopt2`.
* on compare les valeurs de `nb_opt1` et `nb_opt2`, afin de déterminer l'optimum et on renvoie une liste d'activités réalisant cet optimum.

In [35]:
def choix_optimal(L):
    # cas de base 
    if len(L) == 0:
        return []
    # cas général
    a = L[-1] ## on réserve la dernière activité (on rappelle que L[-1] est équivalent à L[len(L) - 1])
    autres_activites = L[:-1] # on rappelle que L[:-1] est équivalent à L[0:len(L) - 1])
    # recherche du  nombre maximal d'activités sans choisir la dernière :
    choix_opt1 = choix_optimal(autres_activites)
    nb_opt1 = len(choix_opt1)
    # recherche du  nombre maximal d'activités si on choisit la dernière :
    compatibles = activites_compatibles(autres_activites, a)
    choix_opt2 = choix_optimal(compatibles)
    nb_opt2 = len(choix_opt2)
    # détermination de l'optimum global et renvoi d'une liste le réalisant
    if nb_opt1 >= nb_opt2 + 1:
        return choix_opt1
    else:
        return choix_opt2 + [a]

* tests

In [36]:
A = [[8., 10.], [9.5, 11.], [10., 10.5], [14., 15.5], [13., 16.]]
res = choix_optimal(A)
print(len(res))
print(res)

3
[[8.0, 10.0], [10.0, 10.5], [14.0, 15.5]]


In [37]:
B = [[9, 12], [13, 15], [16, 21], [11, 13], [14, 18], [8, 14], [15, 19], \
     [8, 9], [13, 17], [20, 22], [12, 16], [15, 17], [10, 21]]
res = choix_optimal(B)
print(len(res))
print(res)

5
[[9, 12], [13, 15], [15, 19], [8, 9], [20, 22]]


***On cherche maintenant à construire, activité par activité, une liste d'activités compatibles, en se donnant un critère, applicable à chaque étape, pour choisir parmi les activités restantes, une activité, compatible avec les activités précédemment choisies, de sorte à maximiser le nombre total d'activités.*** 

**Q3 &emsp; Algorithme glouton pour la sélection d'activités**

**Q3.a** Écrire une fonction `trier_activites` permettant d'ordonner une liste `L` d'activités par heure de fin croissante.  
La fonction prédéfinie `sorted` est autorisée.

In [38]:
help(sorted)

Help on built-in function sorted in module builtins:

sorted(iterable, /, *, key=None, reverse=False)
    Return a new list containing all items from the iterable in ascending order.
    
    A custom key function can be supplied to customize the sort order, and the
    reverse flag can be set to request the result in descending order.



À l'aide de la fonction `sorted`, on peut imposer que le tri se fasse selon les valeurs prises par une fonction `f` lorsqu'on l'applique aux éléments de la liste `L`.  
Par exemple, si on définit :

In [39]:
def f(a):
    return a[0] + a[1]

On peut trier une liste de listes à deux éléments, $[[u_1, v_1], [u_2, v_2], \ldots, [u_n, v_n]]$ selon la valeur des sommes $u_i+v_i$, $1\leqslant i\leqslant n$ :

In [40]:
sorted([[3, 4], [1, 2], [2, 2], [3, 1], [5, 3]], key = f)

***Q3.a corrigé***

In [41]:
def trier_activites(L):
    def f(a):
        return a[1]
    return sorted(L, key = f)

* tests

In [42]:
A = [[8., 10.], [9.5, 11.], [10., 10.5], [14., 15.5], [13., 16.]]
res = trier_activites(A)
print(res)

[[8.0, 10.0], [10.0, 10.5], [9.5, 11.0], [14.0, 15.5], [13.0, 16.0]]


In [43]:
B = [[9, 12], [13, 15], [16, 21], [11, 13], [14, 18], [8, 14], [15, 19], \
     [8, 9], [13, 17], [20, 22], [12, 16], [15, 17], [10, 21]]
res = trier_activites(B)
print(res)

[[8, 9], [9, 12], [11, 13], [8, 14], [13, 15], [12, 16], [13, 17], [15, 17], [14, 18], [15, 19], [16, 21], [10, 21], [20, 22]]


**Q3.b** &emsp; ***\[à faire plus tard, lorsque l'algorithme de tri par insertion aura été vu\]***

Réécrire la fonction `trier_activites` en une fonction `trier_activites_ins` en implémentant un tri par insertion en remplacement de l'appel à la fonction `sorted`.<br/> 
***On prêtera attention dans l'utilisation de `trier_activites_ins` qu'elle implémente un tri en place, alors qu'un appel à la fonction `sorted` produit une nouvelle liste, version triée de la liste en argument. Il faudra en tenir compte selon que l'on utilisera dans la suite l'une ou l'autre de ces deux fonctions.*** 

***Q3.b***

On rappelle une version du tri-insertion en place :

In [44]:
def tri_insertion(L):
    for i in range(1, len(L)):
        aplacer = L[i]
        j = i
        while j > 0 and L[j - 1] > aplacer:
            L[j] = L[j - 1]
            j = j - 1
        L[j] = aplacer

* test

In [45]:
from random import randint
L = [randint(1, 50) for i in range(30)]
print(L)
tri_insertion(L)
print(L)

[35, 15, 5, 22, 10, 18, 7, 3, 50, 10, 41, 32, 9, 38, 23, 35, 47, 30, 43, 10, 18, 4, 34, 31, 38, 33, 11, 22, 7, 35]
[3, 4, 5, 7, 7, 9, 10, 10, 10, 11, 15, 18, 18, 22, 22, 23, 30, 31, 32, 33, 34, 35, 35, 35, 38, 38, 41, 43, 47, 50]


On peut adapter la fonction `tri_insertion` de la façon suivante :

In [46]:
def trier_activites_ins(L):
    for i in range(1, len(L)):
        aplacer = L[i]
        j = i
        while j > 0 and L[j - 1][1] > aplacer[1]:
            L[j] = L[j - 1]
            j = j - 1
        L[j] = aplacer

* test

In [47]:
A = [[8., 10.], [9.5, 11.], [10., 10.5], [14., 15.5], [13., 16.]]
trier_activites_ins(A)
print(A)

[[8.0, 10.0], [10.0, 10.5], [9.5, 11.0], [14.0, 15.5], [13.0, 16.0]]


In [48]:
B = [[9, 12], [13, 15], [16, 21], [11, 13], [14, 18], [8, 14], [15, 19], \
     [8, 9], [13, 17], [20, 22], [12, 16], [15, 17], [10, 21]]
trier_activites_ins(B)
print(B)

[[8, 9], [9, 12], [11, 13], [8, 14], [13, 15], [12, 16], [13, 17], [15, 17], [14, 18], [15, 19], [16, 21], [10, 21], [20, 22]]


**Q3.c** 

Définir une fonction, `selectionner_activite`, prenant en argument deux listes d'activités, `choisies` et `achoisir`, triées par heures de fin croissantes, choisissant au sein de la liste `achoisir` l'activité, **compatible avec la dernière activité de la liste `choisies`** (et donc avec toutes autres les activités de la liste `choisies`), et dont l'heure de fin est minimale.  
La fonction renverra la liste `choisies` à laquelle on aura ajouté l'activité choisie et la liste `achoisir` de laquelle on aura retiré l'activité choisie, ainsi que toutes les activités non compatibles avec l'activité choisie.  
Si aucune activité ne peut être choisie, on renvoie la liste `choisies` sans modification et une liste vide au lieu d'une liste `achoisir`.  

**<u>Attention</u> : on devra traiter le cas où la liste `choisies` est vide.**

***Q3.c corrigé*** 

La liste `achoisir` en entrée étant triée par heure de fin croissante, l'activité à sélectionner est la première de la liste compatible avec la dernière activité de la liste `choisies`, la compatibilité étant assurée si son heure de début est supérieure ou égale à l'heure de fin de la dernière activité de la liste `choisies`

In [49]:
def selectionner_activite(choisies, achoisir):
    if len(choisies) == 0:
        fref = 0
    else:
        fref = choisies[-1][1] ## fref est l'heure de fin de la dernière activité dans la liste choisies
    i = 0
    while i < len(achoisir) and achoisir[i][0] < fref:
        # tant que l'activité de la liste achoisir, achoisir[i], est incompatible avec la dernière activité
        # de la liste choisies, choisies[-1], on passe à l'activité suivante
        i = i + 1
    ## lorsque la boucle while termine :
    ## - si i = len(achoisir), alors aucune activité compatible n'a été trouvée :
    if i < len(achoisir): # on a alors, nécessairement, achoisir[i][0] >= fref
                          # et donc l'activité achoisir[i] est compatible
        # on ajoute l'activité à la liste des activités choisies
        choisies = choisies + [achoisir[i]]
        # on retire de la liste achoisir les activités incompatibles avec l'activité choisie
        ## on ne teste que l'heure de début par rapport à l'heure de fin de achoisir[i]
        compatibles = []
        for activite in achoisir:
            if activite[0] >= achoisir[i][1]:
                compatibles.append(activite)
        achoisir = compatibles
    else:
    ## - sinon, on a i == len(achoisir), et donc aucune activité compatible ne peut être trouvée
        achoisir = []
    return choisies, achoisir

<i><u>note</u> : si on a trié les activités de la liste `achoisir` en utilisant comme première clé de tri l'heure de fin (croissante) et comme seconde clé de tri l'heure de début (croissante), on peut construire la liste `compatibles` sans parcourir toute la liste `achoisir`, puisqu'il suffit de trouver l'indice, $i_0$, de la première activité dans cette liste dont l'heure de début est supérieure ou égale à l'heure de fin de l'activité choisie (on peut donc remplacer la boucle `for activite in achoisir` par une boucle `while` pour déterminer $i_0$, la liste des activités compatibles sera alors donnée par `compatibles = choisies[i_0:]`.</i>

* tests

In [50]:
A = [[8., 10.], [9.5, 11.], [10., 10.5], [14., 15.5], [13., 16.]]
A = trier_activites(A)
print("A triée :")
print(A)
choisies, achoisir = [], A
print("choisies =", choisies)
print("achoisir =", achoisir)
res = selectionner_activite(choisies, achoisir)
print(res[0])
print(res[1])

A triée :
[[8.0, 10.0], [10.0, 10.5], [9.5, 11.0], [14.0, 15.5], [13.0, 16.0]]
choisies = []
achoisir = [[8.0, 10.0], [10.0, 10.5], [9.5, 11.0], [14.0, 15.5], [13.0, 16.0]]
[[8.0, 10.0]]
[[10.0, 10.5], [14.0, 15.5], [13.0, 16.0]]


In [51]:
A = [[8., 10.], [9.5, 11.], [10., 10.5], [14., 15.5], [13., 16.]]
A = trier_activites(A)
print("A triée :")
print(A)
choisies, achoisir = A[:2], A[2:] # équivalent à A[0:2], A[2:len(A)]
print("choisies =", choisies)
print("achoisir =", achoisir)
res = selectionner_activite(choisies, achoisir)
print(res[0])
print(res[1])

A triée :
[[8.0, 10.0], [10.0, 10.5], [9.5, 11.0], [14.0, 15.5], [13.0, 16.0]]
choisies = [[8.0, 10.0], [10.0, 10.5]]
achoisir = [[9.5, 11.0], [14.0, 15.5], [13.0, 16.0]]
[[8.0, 10.0], [10.0, 10.5], [14.0, 15.5]]
[]


In [52]:
B = [[9, 12], [13, 15], [16, 21], [11, 13], [14, 18], [8, 14], [15, 19], \
     [8, 9], [13, 17], [20, 22], [12, 16], [15, 17], [10, 21]]
B = trier_activites(B)
print("B triée :")
print(B)
choisies, achoisir = B[:2], B[2:] # équivalent à B[0:2], B[2:len(B)]
print("choisies =", choisies)
print("achoisir =", achoisir)
res = selectionner_activite(choisies, achoisir)
print(res[0])
print(res[1])

B triée :
[[8, 9], [9, 12], [11, 13], [8, 14], [13, 15], [12, 16], [13, 17], [15, 17], [14, 18], [15, 19], [16, 21], [10, 21], [20, 22]]
choisies = [[8, 9], [9, 12]]
achoisir = [[11, 13], [8, 14], [13, 15], [12, 16], [13, 17], [15, 17], [14, 18], [15, 19], [16, 21], [10, 21], [20, 22]]
[[8, 9], [9, 12], [13, 15]]
[[15, 17], [15, 19], [16, 21], [20, 22]]


**Q3.d**

En réutilisant au mieux ce qui précéde et les fonctions précédemment définies, implémenter l'algorithme glouton de choix d'activités, permettant, à partir d'une liste d'activités, $L$, donnée, de construire une liste d'activités compatibles, maximisant le nombre d'activités de $L$ auxquelles une personne peut assister.  
On implémentera l'algorithme au travers d'une fonction `selection_activites(L)`.  
Cette fonction effectuera un pré-traitement de la liste $L$ consistant à trier les activités par heure de fin croissante.  
Puis, par des appels successifs à la fonction `selectionner_activite`, on construira à partir de la liste $L$ triée, une liste d'activités compatibles à partir d'une liste d'activités choisies vide et d'une liste d'activités à choisir égale à la liste $L$ triée.

In [53]:
def selection_activites(L):
    ...
    

* tester la fonction sur les exemples précédemment examinés

In [54]:
A = [[8., 10.], [9.5, 11.], [10., 10.5], [14., 15.5], [13., 16.]]
res = selection_activites(A)
print(res)

None


In [55]:
B = [[9, 12], [13, 15], [16, 21], [11, 13], [14, 18], [8, 14], [15, 19], \
     [8, 9], [13, 17], [20, 22], [12, 16], [15, 17], [10, 21]]
res = selection_activites(B)
print(res)

None


***Q3.d corrigé***

In [56]:
def selection_activites(L):
    ## pré-traitement
    L = trier_activites(L)
    ## sélection l'une après l'autre des activités
    choisies, achoisir = [], L
    while achoisir != []:
        choisies, achoisir = selectionner_activite(choisies, achoisir)
    return choisies

In [57]:
A = [[8., 10.], [9.5, 11.], [10., 10.5], [14., 15.5], [13., 16.]]
res = selection_activites(A)
print(len(res))
print(res)

3
[[8.0, 10.0], [10.0, 10.5], [14.0, 15.5]]


In [58]:
B = [[9, 12], [13, 15], [16, 21], [11, 13], [14, 18], [8, 14], [15, 19], \
     [8, 9], [13, 17], [20, 22], [12, 16], [15, 17], [10, 21]]
res = selection_activites(B)
print(len(res))
print(res)

5
[[8, 9], [9, 12], [13, 15], [15, 17], [20, 22]]
